# Evaluate VAE reconstruction

In [2]:
import os
from cnn_framework.utils.data_loader_generators.data_loader_generator import (
    DataLoaderGenerator,
)
from cnn_framework.utils.metrics.ssim import SSIM
from cnn_framework.utils.enum import PredictMode
from cnn_framework.utils.data_managers.several_folders_data_manager import (
    SeveralFoldersDataManager,
)

from cell_cycle_classification.backbone.model import FucciVAE
from cell_cycle_classification.utils.data_set import FucciVAEDataSet
from cell_cycle_classification.utils.tools import get_final_model_path
from cell_cycle_classification.utils.vae_model_manager import VAEModelManager
from cell_cycle_classification.utils.euclidean_matching_metric import (
    EuclideanMatchingMetric,
)

from cell_cycle_classification.utils.model_params import FucciVAEModelParams

### Define parameters

In [4]:
params = FucciVAEModelParams()
params.update()

# Define custom paths
params.data_dir = "C:/Users/thoma/data/Data Oriane April/data_repository/all"
params.model_load_path = params.models_folder = "C:/Users/thoma/models/cycle_classification/20241031-112222-4998324"

Model time id: 20250107-160453-local
epochs 10 | batch 32 | lr 0.0001 | weight decay 0.05 | dropout 0.0 | c [0] | z [0, 1, 2, 3, 4] | data set size None | latent dim 256 | beta 0.01 | gamma 100.0 | delta 10000.0 | depth 5 | kld loss standard | encoder name resnet18 | latent dim 256 | beta 0.01 | gamma 100.0 | delta 10000.0 | C 50 | depth 5 | kld loss standard | encoder name resnet18


In [5]:
# Next is specific to our data set - modify with your own data set
params.train_file = os.path.join(os.path.abspath(''), "data_set_split", "train.txt")
params.val_file = os.path.join(os.path.abspath(''), "data_set_split", "val.txt")
params.test_file = os.path.join(os.path.abspath(''), "data_set_split", "test.txt")
params.data_set_size = 280  # maximum nucleus diameter 

### Load data

In [6]:
loader_generator = DataLoaderGenerator(
    params, FucciVAEDataSet, SeveralFoldersDataManager
)
_, _, test_dl = loader_generator.generate_data_loader(shuffle_test=True)

File names correctly loaded.
Splitting file names ...
### Data source ###
train data is loaded from c:\Users\thoma\cell_cycle_classification\notebooks\data_set_split\train.txt - 100% elements
val data is loaded from c:\Users\thoma\cell_cycle_classification\notebooks\data_set_split\val.txt - 100% elements
test data is loaded from c:\Users\thoma\cell_cycle_classification\notebooks\data_set_split\test.txt - 100% elements
###################
train has 752 images.
val has 323 images.
test has 388 images.
###################


### Load pretrained model

In [7]:
final_trained_model = get_final_model_path(params.model_load_path)
model = FucciVAE.load_from_folder(final_trained_model)

### Evaluate reconstruction

In [8]:
VAEModelManager(model, params, SSIM).predict(test_dl);

Current commit hash: 0594c3d0d41c72530f718bb218d8558109c44016
Model evaluation in progress: 100.0% | Batch #12                                                   
Average SSIM: 0.612


### Evaluate time consistency

In [9]:
VAEModelManager(model, params, EuclideanMatchingMetric).predict(
    test_dl, predict_mode=PredictMode.GetEmbeddingMSE
);

Current commit hash: 0594c3d0d41c72530f718bb218d8558109c44016
Model evaluation in progress: 100.0% | Batch #12                                                   
Average EuclideanMatchingMetric: 0.786
